# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'
local_model_name_or_path = '/data/models/chatglm3-6b'

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(local_model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Loading checkpoint shards: 100%|██████████| 7/7 [00:03<00:00,  2.16it/s]


In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(local_model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是八卦之一，也是八宫图中的第一个卦。乾卦是由两个阴爻夹一个阳爻构成，象征着天、云、雷等自然现象，以及君主、领导、权力等象征。乾卦的含义包括创造、领导、坚定、刚毅等。

乾卦的五行属性为木，代表着春季、树木、成长等。在八宫图中，乾卦位于北方，与事业、努力、决断等有关。乾卦的含义与兑卦相对，兑卦代表着喜悦、交流、快乐等。

在八字命理学中，乾卦代表着一个阳刚之气，象征着男性、力量、领导力等。同时，乾卦也代表着一种坚定的信念和决心，以及勇气和自信。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [8]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，也是八宫图中的第二个卦。讼卦由两个阳爻夹一个阴爻构成，象征着打官司、诉讼、争斗等。因此，讼卦的含义包括诉讼、争斗、矛盾、争议等。

讼卦的五行属性为火，代表着夏季、阳光、热情等。在八宫图中，讼卦位于南方，与情感、家庭、婚姻等有关。讼卦与坤卦相对，坤卦代表着顺从、温顺、和谐等。

在八字命理学中，讼卦代表着一种积极的态度和决心，以及一种坚定的信念和决心。同时，讼卦也代表着一种挑战和冒险的精神，以及一种不择手段的决断力。然而，过分刚烈和不懂得妥协，可能会导致诉讼和争斗的不断发生。因此，讼卦也提醒人们要懂得调整自己的态度和行为，以达到和谐、平稳的状态。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [9]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20250524_232457"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [10]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [11]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250524_232457）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [12]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250524_232457）：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


In [13]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
请在提问时提供更多上下文和详细信息，以便我能够更好地回答您的问题。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250524_232457）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [14]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [15]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，具有刚健、积极、勇敢、强悍的特点。它由两个阳爻（代表天）叠加而成，象征着天体雄浑、统一、强大、刚健的特性。在卜问中，乾卦预示着勇敢、刚健的形势，但也提醒要小心谨慎，以避免过于刚健而导致的错误。在好运时，它可以详尽准确地预测未来，但也有着易犯错误的可能。

在哲学上，乾卦代表着刚健、强悍、勇敢、坚定的性格，强调人在在面对困难时，要刚强、坚韧，勇敢地克服困难，迎接挑战。同时，也提醒要尊重天命，顺应形势，才能获得成功。

在解卦时，乾卦的形势为：天<->地

在决策上，乾卦建议在面对问题和困难时，要勇敢、刚健地迎接挑战，积极寻求解决办法。同时，要注重决策的合法性和正统性，避免过于刚健而导致的错误。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天（乾为天）的象形。它由六个阳爻组成，象征着天地的运行和统一。在卜问中，乾卦预示着吉祥，象征着祥瑞和幸福。在解卦时，乾卦代表着天（乾为天）的运行和统一，提示人们要顺应天意，追求和谐。在卜问中，人们可以借助乾卦的吉祥和祥和，祈求好运和幸福。

乾卦的六爻代表着不同的阳爻组合，象征着不同的角色和境遇。每一卦都有不同的象数组合，代表着不同的含义和启示。在解卦时，要结合具体卦象和上下文来理解乾卦的寓意。

在易经中，乾卦预示着天地的运行和统一，提示人们要顺应天意，追求和谐。在解卦时，人们需要根据卦象和上下文来理解乾卦的寓意，从而指导自己的行为和决策。

乾卦的哲学内涵是：顺应天意，追求和谐。在解卦时，要结合具体卦象和上下文来理解乾卦的寓意，从而指导自己的行为和决策。


In [16]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦是周易中的一卦，代表的是地水（坎）之象。在周易中，卦辞描述了这一卦的特性，强调坎水象征 flexible, adaptable, and responsive characteristics, 以及柔顺兼容的优点。

在解卦时，地水师卦预示着前方可能会遇到困难，需要更加谨慎、柔和待人，会有解决问题的可能。在决策方面，需要坚持正道，以观天时、地利、人和，宜取信于人，宜详尽周到，避免冒险。

在卜卦时，需要根据所问问题的具体情况，分析卦象背后的含义，选择合适的解释。在决策过程中，需要根据卦象和卦辞的指引，谨慎、柔和待人，避免冒险，同时注重实际问题中的时机和条件。

总结起来，地水师卦提醒人们，在面对困难和挑战时，需要更加谨慎、柔和待人，以适应不同的情况。在决策过程中，需要注重实际情况，根据时机和条件，决策 Avoid Peril, Follow the Right Path.


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由卦象组成的复卦，由坤上坎下组成，为地水卦。地水卦的含义是：取出水来，取出坤为地，为固定，为顺，为果。取出的水要放在地上，不能乱放，为刚柔相济。

地水师卦的卦象为坤为地，坎为水，表示地上的水需要被约束，为刚柔相济。这个卦象表示，在遵循规则和约束的过程中，柔顺和刚强是必要的。在实际应用中，需要根据实际情况灵活应对，为时机所吸引。

在塔罗牌中，地水师卦代表的是权宜之计，为刚柔相济。在决策时，需要权衡各个方面的情况，根据实际情况灵活应对，为时机所吸引。

在解卦时，地水师卦表示地上的水需要被约束，为刚柔相济。象征着需要有耐心和决心，在困难和挑战中保持冷静和稳定。

地水师卦的核心哲学是：为刚柔相济，需要权宜之计，为时而行。


In [17]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
讼卦是周易卦象中的一枚卦，由上卦坎（水）和下卦乾（天）组成，预示着解决争端的困难和艰辛。在讼卦中，上卦的坎代表险，下卦的乾代表刚，表示虽然双方都有一定力量，但彼此之间存在差异，需要谨慎行事。讼卦提醒人们，在处理争端时，需要善于利用对方的力量，同时保持冷静和耐心，以柔克刚。

讼卦的含义 deepen

讼卦的核心哲学是：

1. 对方可能会对您造成威胁和挑战。
2. 您需要保持冷静和耐心，以柔克刚。
3. 在处理争端时，需要善于利用对方的力量，同时保持谨慎和守卫。
4. 最终，您需要以真诚和宽容的态度，通过谈判或协商来解决争端。

讼卦的启示

讼卦提醒我们，在处理争端时，需要保持冷静和耐心，以柔克刚。在处理争端时，需要善于利用对方的力量，同时保持谨慎和守卫。最终，您需要以真诚和宽容的态度，通过谈判或协商来解决争端。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么吗？ 讼卦是周易卦象中的一枚卦，由上卦坎（水）和下卦乾（天）组成，预示着吉利的变化。在卜筮时，讼卦提醒人们要谨慎待机，以智慧和谋略应对问题，因为事物往往难以预料。

讼卦的核心哲学是：

"天行健，君子以自强不息"

这意味着天空中永远太阳在照，君子应该自强不息，不断求进。在讼卦中，天象征刚健，而刚健的刚象征刚健，刚健的义指刚强有力，因此刚健的刚象征刚强有力。刚强有力可以依赖自己的力量，也可以依赖别人的力量，但依赖总不是长久之计。

讼卦的时运是：

初爻：刚强有力

二爻：刚直不阿

三爻：刚强有力

四爻：刚强有力

讼卦的运势是：

初爻刚强有力，但容易引起冲突，刚直不阿，不善于团结，容易惹怒他人，因此需要谨慎行事，初初时可能并不顺利。

二爻刚直不阿，但缺乏耐心，容易引起冲突，刚强有力，但缺乏耐心，需要收敛自己的性格，初初时可能并不顺利。

三爻刚强有力，但过于刚强，容易引起冲突，刚强有力，但过于刚强会容易引起冲突，因此需要强硬不屈，强硬不阿。

四爻刚强有力，但能够宴请朋友，刚强有力，但能够宴请朋友，因此需要强硬不屈，强硬不阿。

讼卦的启示是：

刚强不屈，刚强不阿，直则观其初，待机待势，求得吉祥。

讼卦的时运是：

初爻：初入社会，刚强有力，但容易引起冲突，刚直不阿

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [18]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [19]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是八卦之一，属于阳刚之气。乾卦象征着天、云、高山等，以及龙、丽、权威、强盛等，代表着强大、威武、刚健、刚毅、正义等。在八卦中，乾卦位于北方，与事业、权力、法律、权威等有关，象征着强大的阳刚之气。

乾卦的卦象是由两个乾卦上下相叠而成，象征着刚强、威武、权威、强大等。乾卦的卦象是由六阳爻构成，阳爻代表阳刚之气，阴爻代表阴柔之气。乾卦表示阳刚之气，要求强阳之极，象征著刚毅、正义、强大等。

在周易中，乾卦代表阳、强、威、顺，具有权威和正义的象征。它表示在事业、权力等方面具有权威和正义的象征，但也不能过于刚毅，需要适度调整。乾卦的卦象象征着高、权威、强大，可以象征一元复始，也可以象征万物归一。

乾卦的卦象和卦义具有强大的象征意义，可以用来象征正义、权威、强大、刚毅、正义等。在五行中，乾卦代表金，象征着刚强、权威、强大。在卜卦中，乾卦通常象征顺从、恭维、逢迎，表示适度调整，不要过于刚毅。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 解释下乾卦是什么？乾卦是《易经》中的第一卦，也是六十四卦中的首卦。乾卦象征着天，代表着刚强、强大、积极向上、积极进取，以及正义和公平。乾卦由三个阳爻和一个阴爻构成，象征着阳与阴的结合，也象征着男女的结合。在《易经》中，乾卦代表着“天”，是卦象中阴爻夹阳爻，阳爻代表阳刚，阴爻代表阴柔。乾卦的卦辞是：“元：大，始。亨：通，顺利。利：有利，顺遂。益：受益，增强。” 乾卦象征着天，也象征着刚强和正义，常用来象征父亲和阳刚之德。


In [20]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
师卦是《易经》中的第八卦，卦象为：上卦为坎，坎为水；下卦为师，师代表师众，象征军队。师卦象征军队众，兵败师散，宜及时止战，众兵合围，众志成城，得利。

师卦的卦辞：师，众，众人，象征师众，战师，兵众势众，得利。

师卦的卦辞：师，众，师众象征众，兵败众散，需时整合，宜止战，得利。

师卦的卦辞：师，众，象征众，师众，战师，众志成城，得利。

师卦的卦辞：师，众，师众，象征众，兵众势众，宜时整合，得利。

师卦的卦辞：师，众，师众，象征众，兵败众散，时众整合，得利。

师卦的卦辞：师，众，师众，象征众，兵众势众，宜时整合，得利。

师卦的卦辞：师，众，师众，象征众，战师，众志成城，得利。

师卦卦辞解读：师卦坎上坤下，象征众，师众，宜时整合，得利。师卦坎象征水众，象征师众，战师，众志成城，得利。上坎下坤，众宜时整合，得利。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦 师卦原文：师。贞，丈人吉，无咎。象曰：地中有水，师。君子以容民畜众。白话文解释：师卦象征军队指挥，无灾祸。《象辞》说：下卦为坎（水），上卦为坤（地），如大地容纳江河，君子应容纳众人。《断易天机》解：师卦坤上坎下，象征军众，需德高长者统率以吉无咎。北宋易学家邵雍解：忧劳动众，公正无私排难。得卦者应包容他人，努力排除困难。台湾国学大儒傅佩荣解：时运包容他人，财运有财需珍惜，家宅旧亲联姻吉，身体腹胀调气。传统解卦：异卦（下坎上坤），“师”指军队。坎为水险，坤为地顺，寓兵于农，用兵应顺势，故化凶为吉。


In [21]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
讼卦（卦名：讼）震卦下，巽卦上，卦象：上卦为巽卦，下卦为坎卦。坎卦为水，象征险阻，喻诉讼诉讼之事难以解决，需谨慎小心。

讼卦象征诉讼之事，卦象是下卦坎卦，上卦巽卦。坎卦象征险阻，喻诉讼诉讼之事难以解决，需谨慎小心。

 Contact: 33880


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 天水讼卦 讼卦原文：讼。有孚，窒惕，中吉，终凶。利见大人，不利涉大川。象曰：天与水违行，讼。君子以做事谋始。白话文解释：讼卦象征虽有利可图但需警惕。事情初吉后凶，利于见贵人，不宜涉水。《象辞》说：上卦为乾（天），下卦为坎（水），天水相隔，事理不合，君子需慎重谋事。《断易天机》解：讼卦乾上坎下，刚遇险，必有争论，多不吉。北宋易学家邵雍解：天高水深，远离不亲，慎谋退守则无凶。得此卦者，身心不安，多争诉，宜修身养性。台湾国学大儒傅佩荣解：时运受阻，财运初谨慎终获利，家宅君子求淑女，身体预防胜于治疗。传统解卦：异卦（下坎上乾），刚健遇险，彼此反对，生争讼，需慎重戒惧。
